In [2]:
import altair as alt, pandas as pd, requests, json

In [3]:
def get_ons_data(series_id):
    dataset_id='MGDP'

    # Use ONS API to get monthly data
    url = f'https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/{series_id}/dataset/{dataset_id}/data'
    # read data at api into dataframe, relevant data is in the 'months' key
    # Make a GET request to fetch the raw JSON content
    json_data = requests.get(url).json()

    # Extract data from the 'months' key
    months_data = json_data['months']

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame.from_dict(months_data)

    # Clean data set, convert date from yyyy mmm to yyyy-mm-dd
    df['date'] = pd.to_datetime(df['date'], format='%Y %b')

    # drop unnecessary columns
    df.drop(columns=['label', 'quarter', 'sourceDataset', 'updateDate'], inplace=True)

    # convert value column to float
    df['value'] = df['value'].astype(float)

    return df

In [3]:
df_ecy2 = get_ons_data(series_id='ecy2')

In [4]:
df_ecy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    320 non-null    datetime64[ns]
 1   month   320 non-null    object        
 2   value   320 non-null    float64       
 3   year    320 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.1+ KB


In [4]:
dark = True

background = '#122b39' if dark else '#b4c8d8'
detail = '#b4c8d8' if dark else '#122b39'
line_colour = '#36b7b4'

In [6]:
# create chart using Altair

chart = alt.Chart(df_ecy2[df_ecy2['date'] >= '2006-01-01']).mark_line(color=line_colour).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='Monthly index, 2019 = 100 | Source: ONS',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        ),
        scale=alt.Scale(zero=False, padding=45)
    )
).properties(
    width = 400,
    height = 300,
    title = 'UK GDP'
)

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ecy2[df_ecy2['date'] == df_ecy2['date'].max()]

point = alt.Chart(last_point).mark_point(color=line_colour, size=60, opacity=0.9, filled=True).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q')
)

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}')

text_value = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=0,
    color=line_colour,
    fontSize=14,
).encode(
    text='text'
)

text_month = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=-16,
    color=line_colour,
    fontSize=14,
).encode(
    text='month'
)

chart = chart + point + text_month + text_value


chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=20,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_26001/1003967247.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}')


alt.LayerChart(...)

---

Gross Value Added - Monthly (period on period growth): CVM SA

In [139]:
# Monthly (period on period growth)
df_ecyx = get_ons_data(series_id='ecyx')

# Gross Value Added - Monthly (3 month on 3 month growth) :CVM SA
df_ed3h = get_ons_data(series_id='ed3h')

In [209]:
bar_positive = '#36b7b4'
bar_neagtive = '#e6224b'

In [222]:
# Create bar chart of recent monthly GDP growth

chart = alt.Chart(df_ecyx[df_ecyx['date'] >= '2022-01-01']).mark_bar(width=13).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        ),
        scale=alt.Scale(padding=20)
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=7,
            tickOpacity=0.5,
            ticks=False,
            title='Monthly GDP Growth, % | Source: ONS',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        ),
        scale=alt.Scale(zero=False, padding=35)
    ),
    # conditionally set colour of bar positive or negative value
    color=alt.condition(
        alt.datum.value > 0,
        alt.value(bar_positive),
        alt.value(bar_neagtive)
    )
).properties(
    width = 400,
    height = 300,
    title = 'UK GDP Growth'
)

######

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ecyx[df_ecyx['date'] == df_ecyx['date'].max()]

# text_value = alt.Chart(last_point).mark_point(color=bar_positive, size=60, opacity=0.9, filled=True).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q')
# )

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')

text_value = alt.Chart(last_point).mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-9,
    color=detail,
    fontSize=13,
).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q'),
    text='text'
)

text_month = text_value.mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-25,
    color=detail,
    fontSize=13,
).encode(
    text='month'
)

########

# add a line at y=0
chart += alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color=detail, size=1.5, opacity=0.7).encode(y='y')

chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=23,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()


/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_10342/2793686299.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')


alt.LayerChart(...)

In [223]:
## add text

chart + text_month + text_value

alt.LayerChart(...)

In [224]:
# Add overlay line chart of 3 month on 3 month growth
line_colour = '#fff'
chart_line = alt.Chart(df_ed3h[df_ed3h['date'] >= '2022-01-01']).mark_line(color=line_colour, strokeDash=[4,2], strokeOpacity=0.5).encode(
    x='date:T',
    y='value:Q'
)

chart += chart_line

In [226]:
chart.display()

alt.LayerChart(...)

---

Gross Value Added - Monthly (period on period 1 year ago growth ) :CVM SA

In [5]:
df_ed2r = get_ons_data(series_id='ed2r')

In [6]:
# create chart using Altair

chart = alt.Chart(df_ed2r[df_ed2r['date'] >= '2013-01-01']).mark_line(color=line_colour).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='Gross value added, 12-month growth, % | Source: ONS',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        ),
        scale=alt.Scale(zero=False, padding=30)
    )
).properties(
    width = 400,
    height = 300,
    title = 'UK GDP Growth'
)

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df_ed2r[df_ed2r['date'] == df_ed2r['date'].max()]

point = alt.Chart(last_point).mark_point(color=line_colour, size=60, opacity=0.9, filled=True).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q')
)

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')

text_value = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=0,
    color=line_colour,
    fontSize=14,
).encode(
    text='text'
)

text_month = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=-16,
    color=line_colour,
    fontSize=14,
).encode(
    text='month'
)

chart = chart + point + text_month + text_value

# add a line at y=0
chart += alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color=detail, size=1.5, opacity=0.5).encode(y='y')


chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=20,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()



/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_32519/522539290.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1f}%')


alt.LayerChart(...)

In [7]:
# Convert chart to dictionary
chart_dict = chart.to_dict()

# Modify width and height directly (Altair will set continuousWidth and height by default)
chart_dict['width'] = 350
chart_dict['height'] = 280

# Convert the dictionary back to JSON
vega_spec = json.dumps(chart_dict, indent=2)

# Write the JSON to a file
with open('../../charts/202310/20231012_UK_GDP_Growth_12month.json', 'w') as f:
    f.write(vega_spec)

---